In [1]:
import os
import glob
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# import upsetplot
#import humanize

pd.set_option('display.max_rows', 500)

import pybedtools as pbt 
pbt.set_bedtools_path('/mnt/BioHome/jreyna/software/anaconda3/envs/hic_tls/bin/')
#pbt.set_tempdir('/mnt/bioadhoc-temp/Groups/vd-ay/jreyna-temp/scratch/')


os.chdir('/mnt/bioadhoc-temp/Groups/vd-ay/jreyna/projects/t1d-loop-catalog/')
outdir = 'results/hg38/finemapping/sgls/summary/'
os.makedirs(outdir, exist_ok=True)

In [2]:
# 35 different sample names
fns = glob.glob('results/hg38/finemapping/sgls/*.5000/*.5000.finemap_sgls.tsv')

In [65]:
# each sample has biological duplicates
for fn in fns:
    if "CD8_Naive_1815-RH-1" in fn:
        print(fn)

results/hg38/finemapping/sgls/CD8_Naive_1815-RH-1.phs001703v3p1.Homo_Sapiens.H3K27ac.b2.5000/CD8_Naive_1815-RH-1.phs001703v3p1.Homo_Sapiens.H3K27ac.b2.5000.finemap_sgls.tsv
results/hg38/finemapping/sgls/CD8_Naive_1815-RH-1.phs001703v3p1.Homo_Sapiens.H3K27ac.b1.5000/CD8_Naive_1815-RH-1.phs001703v3p1.Homo_Sapiens.H3K27ac.b1.5000.finemap_sgls.tsv
results/hg38/finemapping/sgls/CD8_Naive_1815-RH-1.phs001703v3p1.Homo_Sapiens.H3K27ac.b3.5000/CD8_Naive_1815-RH-1.phs001703v3p1.Homo_Sapiens.H3K27ac.b3.5000.finemap_sgls.tsv


In [3]:
# exmaple of treating one fn
sample = fns[0]
sample_name = (sample.split("/")[-1]).split(".")[0]
df = pd.read_table(sample)
df["sample_name"] = sample_name
df["summary_id"] = df["chr_snp"].astype(str) + "_" + df["start_snp"].astype(str) + "_" \
                    + df["geneid"].astype(str) + "_" + df["startA_loop"].astype(str) + "_" \
                    + df["startB_loop"].astype(str)
df

In [5]:
# treating all fns
dfs = []
for fn in fns:
    sample_name = (fn.split("/")[-1]).split(".")[0]
    df = pd.read_table(sample)
    df["sample_name"] = sample_name
    df["summary_id"] = df["chr_snp"].astype(str) + "_" + df["start_snp"].astype(str) + "_" \
                        + df["geneid"].astype(str) + "_" + df["startA_loop"].astype(str) + "_" \
                        + df["startB_loop"].astype(str)
    dfs.append(df)
    
all_df = pd.concat(dfs, ignore_index=True)

In [6]:
all_df

,chrA_loop,startA_loop,endA_loop,chrB_loop,startB_loop,endB_loop,-log10_qval_loop,chr_snp,start_snp,end_snp,...,ppa_snp,snp_anchor,chr_gene,start_gene,end_gene,genename,geneid,strand,sample_name,summary_id
0,1,112615000,112620000,1,113905000,113910000,6.195724,1,113909957,113909958,...,0.058538,AnchorB,1,112619804,112619805,CAPZA1,ENSG00000116489,+,CD8_Naive_1815-RH-1,1_113909957_ENSG00000116489_112615000_113905000
1,1,113905000,113910000,1,113925000,113930000,18.295846,1,113909957,113909958,...,0.058538,AnchorA,1,113929323,113929324,HIPK1,ENSG00000163349,+,CD8_Naive_1815-RH-1,1_113909957_ENSG00000163349_113905000_113925000
2,1,113905000,113910000,1,113925000,113930000,18.295846,1,113909957,113909958,...,0.058538,AnchorA,1,113929491,113929492,HIPK1-AS1,ENSG00000235527,-,CD8_Naive_1815-RH-1,1_113909957_ENSG00000235527_113905000_113925000
3,11,128295000,128300000,11,128520000,128525000,11.801121,11,128298735,128298736,...,0.130943,AnchorA,11,128522389,128522390,MIR6090,ENSG00000276176,+,CD8_Naive_1815-RH-1,11_128298735_ENSG00000276176_128295000_128520000
4,11,128300000,128305000,11,128520000,128525000,53.208704,11,128302574,128302575,...,0.122752,AnchorA,11,128522389,128522390,MIR6090,ENSG00000276176,+,CD8_Naive_1815-RH-1,11_128302574_ENSG00000276176_128300000_128520000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6358,6,90275000,90280000,6,90295000,90300000,12.402266,6,90279405,90279406,...,0.083274,AnchorA,6,90296907,90296908,BACH2,ENSG00000112182,-,Monocyte_1831-RH-1,6_90279405_ENSG00000112182_90275000_90295000
6359,6,158650000,158655000,6,159040000,159045000,6.867517,6,159044944,159044945,...,0.122922,AnchorB,6,158650013,158650014,SYTL3,ENSG00000164674,+,Monocyte_1831-RH-1,6_159044944_ENSG00000164674_158650000_159040000
6360,6,159040000,159045000,6,159090000,159095000,8.984821,6,159044944,159044945,...,0.122922,AnchorA,6,159094092,159094093,AL356417.1,ENSG00000224478,+,Monocyte_1831-RH-1,6_159044944_ENSG00000224478_159040000_159090000
6361,6,159040000,159045000,6,159095000,159100000,26.156556,6,159044944,159044945,...,0.122922,AnchorA,6,159095822,159095823,AL035530.1,ENSG00000226032,-,Monocyte_1831-RH-1,6_159044944_ENSG00000226032_159040000_159095000


In [9]:
# all_df.to_csv(outdir + 'all_sgls.tsv', index=False, sep="\t")

In [15]:
# Group lines with the same key
grouped_df = all_df.groupby('summary_id').agg({'sample_name': ', '.join, 
                                               '-log10_qval_loop':'min',
                                               'ppa_snp':'max',
                                              }).reset_index()

In [16]:
# 35 samples
grouped_df

,summary_id,sample_name,-log10_qval_loop,ppa_snp
0,11_128298735_ENSG00000276176_128295000_128520000,"CD8_Naive_1815-RH-1, Naive_K_1829-RH-1, CD4_Na...",11.801121,0.130943
1,11_128302574_ENSG00000254588_128300000_128525000,"CD8_Naive_1815-RH-1, Naive_K_1829-RH-1, CD4_Na...",12.543043,0.122752
2,11_128302574_ENSG00000276176_128300000_128520000,"CD8_Naive_1815-RH-1, Naive_K_1829-RH-1, CD4_Na...",53.208704,0.122752
3,11_128302747_ENSG00000254588_128300000_128525000,"CD8_Naive_1815-RH-1, Naive_K_1829-RH-1, CD4_Na...",12.543043,0.125901
4,11_128302747_ENSG00000276176_128300000_128520000,"CD8_Naive_1815-RH-1, Naive_K_1829-RH-1, CD4_Na...",53.208704,0.125901
5,11_128304885_ENSG00000254588_128300000_128525000,"CD8_Naive_1815-RH-1, Naive_K_1829-RH-1, CD4_Na...",12.543043,0.127477
6,11_128304885_ENSG00000276176_128300000_128520000,"CD8_Naive_1815-RH-1, Naive_K_1829-RH-1, CD4_Na...",53.208704,0.127477
7,11_128308902_ENSG00000276176_128305000_128520000,"CD8_Naive_1815-RH-1, Naive_K_1829-RH-1, CD4_Na...",7.669259,0.116469
8,12_56040361_ENSG00000135482_56040000_56115000,"CD8_Naive_1815-RH-1, Naive_K_1829-RH-1, CD4_Na...",7.437393,0.350915
9,12_56040361_ENSG00000139641_56040000_56115000,"CD8_Naive_1815-RH-1, Naive_K_1829-RH-1, CD4_Na...",7.437393,0.350915


In [39]:
# Test if I group_by worked
test_str = grouped_df["sample_name"][1]
test_ls = test_str.split(", ")
test_set = set(test_ls)
print(test_ls)

['CD8_Naive_1815-RH-1', 'Naive_K_1829-RH-1', 'CD4_Naive_1816-RH-1', 'CD8_Naive_1800-RH-1', 'CD8_Naive_1831-RH-1', 'CD8_Naive_1814-RH-1', 'Nonclassical_Monocyte_1831', 'CD4_Naive_1815-RH-1', 'CD4_Naive_1829-RH-1', 'Nonclassical_Monocyte_1786', 'Naive_B_1829-RH-1', 'CD8_Naive_1800-RH-1', 'Naive_K_1829-RH-1', 'Monocyte_1814-RH-1', 'Nonclassical_Monocyte_1831', 'Monocyte_1831-RH-1', 'Naive_K_1816-RH-1', 'Naive_B_1814-RH-1', 'Monocyte_1800-RH-1', 'CD8_Naive_1815-RH-1', 'CD4_Naive_1800-RH-1', 'Nonclassical_Monocyte_1800', 'Monocyte_1800-RH-1', 'Naive_K_1831-RH-1', 'Nonclassical_Monocyte_1786', 'Nonclassical_Monocyte_1800', 'CD4_Naive_1829-RH-1', 'CD4_Naive_1831-RH-1', 'Monocyte_1829-RH-1', 'Naive_B_1816-RH-1', 'Naive_K_1800-RH-1', 'Nonclassical_Monocyte_1786', 'Naive_B_1800-RH-1', 'Monocyte_1831-RH-1', 'CD8_Naive_1800-RH-1', 'Naive_K_1815-RH-1', 'CD4_Naive_1816-RH-1', 'CD4_Naive_1831-RH-1', 'CD4_Naive_1814-RH-1', 'Monocyte_1816-RH-1', 'CD4_Naive_1816-RH-1', 'Naive_K_1815-RH-1', 'CD8_Naive_18

In [41]:
answer_set = set(all_df[all_df["summary_id"] == "11_128302574_ENSG00000254588_128300000_128525000"]["sample_name"])

In [42]:
test_set - answer_set, answer_set - test_set

(set(), set())